In [5]:
%cd ..

c:\Users\HP\OneDrive - University of Moratuwa\Desktop\E-Vision-Projects\Shelf_Product_Count_Generation


In [1]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
import base64
from pathlib import Path
from dotenv import load_dotenv
import os

In [2]:
_ = load_dotenv() 

In [3]:
llm = ChatOpenAI(
            model='gpt-4o-mini',
            api_key=os.getenv("OPENAI_API_KEY")
        )

llm.invoke('Hi')

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_aa07c96156', 'id': 'chatcmpl-Cl9uD32hVIt6n8e5odcpYKRx033hZ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--2459d418-dbb3-4466-82df-209df72fb8cf-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [26]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
import base64

# Define Pydantic model for structured output
class Product(BaseModel):
    brand: str = Field(description="Extract the exact brand name from visible labels or logos")
    product_name: str = Field(description="Full product name or title as shown on the packaging")
    variant: str = Field(description="Identify the variant or type")
    primary_colors: list[str] = Field(description="List the dominant colors of the packaging")
    packaging_type: str = Field(description="Type of container (e.g., 'Bottle', 'Tube', 'Jar', 'Box', 'Can')")
    shape: str = Field(description="Describe the container shape (e.g., 'Cylindrical', 'Rectangular', 'Oval')")
    size_volume: str = Field(description="Extract exact size/volume from label (e.g., '200ml', '500g') or estimate if not visible")
    logo_description: str = Field(description="Brief description of the brand logo appearance and position.")
    key_text: list[str] = Field(description="Extract main visible text or claims on the label")
    distinctive_elements: list[str] = Field(description="Any unique design features, patterns, or color combinations that make this product recognizable")

class ProductInfo(BaseModel):
    product: Product

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


def get_product_info(base64_image):
    parser = JsonOutputParser()
    
    prompt = """
    Analyze the product image and extract key identifying features.
    
    {format_instructions}
    
    Extract: brand, product_name, variant, primary_colors (list), packaging_type, 
    shape, size_volume, logo_description, key_text (list), distinctive_elements (list).
    
    Mark unclear features as "Unknown".
    """
    
    format_instructions = parser.get_format_instructions()
    prompt = prompt.format(format_instructions=format_instructions)
    
    message = HumanMessage(
        content=[
            {"type": "text", "text": prompt},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
        ]
    )
    
    response = llm.invoke([message])
    result = parser.parse(response.content)
    
    return result

In [27]:

image_path = 'data/test_images/cropped_image_IMG_2320_9.jpg'
base64_image = encode_image(image_path)
result = get_product_info(base64_image)

In [28]:
result

{'brand': 'Janet',
 'product_name': 'Hair Oil',
 'variant': 'Dark Henna',
 'primary_colors': ['Red', 'Green', 'Pink'],
 'packaging_type': 'Box',
 'shape': 'Rectangular',
 'size_volume': 'Unknown',
 'logo_description': 'Janet Ayurveda in cursive',
 'key_text': ['HAIR FALL CONTROL', 'DARK HENNA', 'HAIR OIL', 'PARABEN FREE'],
 'distinctive_elements': ['Ayurveda branding',
  'Images of natural ingredients']}

In [31]:
reference_image_path = 'data/reference_images'
product_database = {}

for product in os.listdir(reference_image_path):
    product_path = os.path.join(reference_image_path,product)
    image_name = os.listdir(product_path)[0]
    image_path = os.path.join(product_path, image_name)
    
    base64_image = encode_image(image_path)
    product_info = get_product_info(base64_image)
    
    product_database[product] = product_info
    
    print(f"Processed: {product}")

Processed: 1000
Processed: 1001
Processed: 1002
Processed: 1003
Processed: 1004
Processed: 1005
Processed: 1006
Processed: 1007
Processed: 1008
Processed: 1009
Processed: 1010
Processed: 1011
Processed: 1012
Processed: 1013
Processed: 1014
Processed: 1015
Processed: 1016
Processed: 1017
Processed: 1018
Processed: 1019
Processed: 1020


In [35]:
# After product_database is created
for idx, (product_key, product_data) in enumerate(product_database.items(), start=1):
    product_data['product_id'] = f"{product_key}"
product_database

{'1000': {'brand': 'emami',
  'product_name': '7 OILS IN ONE',
  'variant': 'HAIR OIL',
  'primary_colors': ['Orange', 'Gold', 'White'],
  'packaging_type': 'Bottle',
  'shape': 'Unknown',
  'size_volume': '100 ml',
  'logo_description': 'Unknown',
  'key_text': ['PARABEN FREE',
   'BLENDS FOR DAMAGE CONTROL',
   'Dry & Brittle Hair',
   'SHEA BUTTER'],
  'distinctive_elements': ['7 oils listed: Argan, Avocado, Aloe Vera, Amla, Coconut, Walnut'],
  'product_id': '1000'},
 '1001': {'brand': 'Emami',
  'product_name': '7 Oils in One Hair Oil',
  'variant': 'Nourishing Aloe Vera',
  'primary_colors': ['Green', 'Gold', 'White'],
  'packaging_type': 'Bottle',
  'shape': 'Round',
  'size_volume': 'Unknown',
  'logo_description': 'Unknown',
  'key_text': ['PARABEN FREE',
   'HAIR OIL',
   'Hair Fall Control',
   'NOURISHING ALOE VERA',
   'BLENDS FOR DAMAGE CONTROL'],
  'distinctive_elements': ['7 Oils in One', 'Aloe Vera Illustration'],
  'product_id': '1001'},
 '1002': {'brand': 'Unknown',


In [37]:
import json
with open('data/product_reference_database.json', 'w') as f:
    json.dump(product_database, f, indent=2)